In [ ]:
# make local editable packages automatically reload
%load_ext autoreload
%autoreload 2
    

In [ ]:

# Import dependencies
import numpy as np
import omnipose

# set up plotting defaults
from omnipose.plot import imshow
omnipose.plot.setup()

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
from omnipose.gpu import use_gpu
use_GPU = use_gpu()


In [ ]:
from pathlib import Path
import os, re
from cellpose_omni import io
from cellpose_omni.io import imread, imwrite

omnidir = Path(omnipose.__file__).parent.parent.parent
basedir = os.path.join(omnidir,'docs','_static')

imgs = imread(os.path.join(basedir, 'long_10_2_phase.tif'))

In [ ]:
imshow([imgs[i] for i in range(0,len(imgs),10)])

In [ ]:
import cellpose_omni
from cellpose_omni import models

model = models.CellposeModel(gpu=use_GPU, 
                             model_type='bact_phase_affinity', 
                             nclasses=2, nchan=1, 
                             # nsample=nsample,
                             # dilation = dilation
                            )
chans = None


In [ ]:
# parallelization using dataloader workflow

eval_set = omnipose.data.eval_set(imgs,dim=2, device=model.device, extra_pad = 1,
                                  pad_mode='replicate', 
                                  # replicate is a really important thing to do for fast time lapses, 
                                  #exponential growth can cause thin features 
                                 )
params = {'channels':chans,
          'rescale': 1,
          # 'indices':[0],
          # 'show_progress': True,
          'mask_threshold': -1,
          'num_workers':0,
          'transparency': True,
          'flow_threshold': 0,
          'omni': True,
          # 'cluster': 1,
          'verbose':0,
          # 'affinity_seg': 1,
          # 'tile': 1,
          # 'min_size': 9,
          # 'compute_masks':0,
          # 'niter': None 
          # 'niter':10 
        }

from omnipose.gpu import empty_cache
empty_cache()

masks, flows, _ = model.eval(eval_set,**params)
masks.shape, imgs.shape

In [ ]:
# the usual per-image workflow

empty_cache()
masks, flows, _ = model.eval([i for i in imgs],**params)

In [ ]:
len(flows), len(masks)

In [ ]:
from cellpose_omni import plot
import omnipose

n = [0,10,len(imgs)-1]

for idx,i in enumerate(n):
    print(idx, i)
    maski = masks[i] # get masks
    bdi = flows[i][-1] # get boundaries
    flowi = flows[i][0] # get RGB flows 

    # set up the output figure to better match the resolution of the images 
    f = 15
    szX = maski.shape[-1]/mpl.rcParams['figure.dpi']*f
    szY = maski.shape[-2]/mpl.rcParams['figure.dpi']*f
    fig, ax = omnipose.plot.figure(figsize=(szY,szX*4), facecolor=[0]*4, frameon=False)
    
    plot.show_segmentation(fig, omnipose.utils.normalize99(imgs[i]), 
                           maski, flowi, bdi, channels=None, omni=True,
                           interpolation=None)

    plt.tight_layout()
    plt.show()